In [1]:
import pandas as pd
import seaborn as sns
import statsmodels
import pylab as plt
import numpy as np

In [2]:
plt.style.use('bmh')

In [3]:
import statsmodels.formula.api as sm

In [4]:
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import FuncTickFormatter, LabelSet, Label, ColumnDataSource, TextAnnotation
from bokeh.models import HoverTool, ColorBar
from bokeh.models.mappers import LinearColorMapper
from bokeh.palettes import Inferno256
from bokeh.io import show, push_notebook, output_notebook, output_file, curdoc
from bokeh.models.widgets import Panel, Tabs
from bokeh.models.widgets import Select
from bokeh.layouts import column
output_notebook()

Loading BokehJS ...

In [5]:
def RAD(deg):
    return deg * np.pi / 180.0

def GET_R2(data_lv, num_seg):
    df = data_lv.loc[data_lv.Segm == num_seg]
    result = sm.ols(formula="Voltage ~ WT", data=df).fit()
    return result.rsquared


def GET_SLOPE(data_lv, num_seg):
    df = data_lv.loc[data_lv.Segm == num_seg]
    result = sm.ols(formula="Voltage ~ WT", data=df).fit()
    return result.params.WT

In [6]:
output_file("bv90.html")

tab_ls = list()

for case in ['bv90_lv', 'bv90_2p', 'bv90_s20', 'bv90_s100']:
    mapper = LinearColorMapper(palette=Inferno256, low=0.0, high=1.0)
    color_bar = ColorBar(color_mapper=mapper, border_line_color=None, location=(0,0))
    p = figure(title='Statistics for segments', plot_width=500, plot_height=700, x_range=(-10,10), y_range=(-10,10), tools="save")
    p.add_layout(color_bar, 'right')
    p.toolbar.logo = None    
    data = pd.read_csv('data/{}.csv'.format(case), index_col=0)
    
    s_apex = ColumnDataSource(data=dict(name=['17'], img=['plots/{}/17.png'.format(case)], r2=[GET_R2(data, 17)], slope=[GET_SLOPE(data, 17)]))

    s_apical = [ColumnDataSource(data=dict(
        name=['{}'.format(12 + i)],
        img=['plots/{}/{}.png'.format(case, 12 + i)],
        r2=[GET_R2(data, 12 + i)],
        slope=[GET_SLOPE(data, 12 + i)])) for i in [4, 1, 2, 3]]

    s_basal_1 = [ColumnDataSource(data=dict(
        name=['{}'.format(6 + i)], img=['plots/{}/{}.png'.format(case, 6 + i)],
        r2=[GET_R2(data, 6 + i)],
        slope=[GET_SLOPE(data, 6 + i)])) for i in [6, 1, 2, 3, 4, 5]]

    s_basal_2 = [ColumnDataSource(data=dict(
        name=['{}'.format(i)],
        img=['plots/{}/{}.png'.format(case, i)],
        r2=[GET_R2(data, i)],
        slope=[GET_SLOPE(data, i)])) for i in [6, 1, 2, 3, 4, 5]]
            
    r0 = 2
    rd = 2
    p.axis.visible = False
    p.ellipse([0],[0],width=20,height=9.5,height_units='data', width_units='data',fill_color='white',line_width=6)
    p.patch([0,10,10,0,0],[10,10,-10,-10,10],fill_color='white',line_color='white')
    #p.wedge(x=0, y=0, radius=r0, start_angle=RAD(0), end_angle=RAD(360), color={'field': 'r2', 'transform': mapper}, alpha=0.6, line_width=2, source=s_apex)
    p.circle(x=0, y=0, size=80, color={'field': 'r2', 'transform': mapper}, alpha=0.6, line_width=2, source=s_apex)
    for i in range(0,4):
        p.annular_wedge(x=0, y=0, inner_radius=r0, outer_radius=r0+rd, start_angle=RAD(-45+i*90), end_angle=RAD(45+i*90), 
                        color={'field': 'r2', 'transform': mapper}, alpha=0.6, line_width=2,
                       source=s_apical[i])
    r0 += rd
    for i in range(6):
        p.annular_wedge(x=0, y=0, inner_radius=r0, outer_radius=r0+rd, start_angle=RAD(0+i*60), end_angle=RAD(60+i*60), 
                        color={'field': 'r2', 'transform': mapper}, alpha=0.6, line_width=2,
                       source=s_basal_1[i])
    r0 += rd
    for i in range(6):
        p.annular_wedge(x=0, y=0, inner_radius=r0, outer_radius=r0+rd, start_angle=RAD(0+i*60), end_angle=RAD(60+i*60), 
                        color={'field': 'r2', 'transform': mapper}, alpha=0.6, line_width=2,
                       source=s_basal_2[i])

    p.add_tools(HoverTool(attachment="right",
        tooltips="""
        <div>
            <div>
                <span style="font-size: 12;"><b>Segment</b>: @name</span><br />
            </div>
            <div>
                <img
                    src="@img" height="350" alt="@img" width="250"
                    style="float: left; margin: 0px 15px 15px 0px;"
                ></img>
            </div>
            <div>
                <span style="font-size: 12;"><b>R2</b>: @r2</span><br />
                <span style="font-size: 12;"><b>slope</b>: @slope</span><br />
            </div>
        </div>
        """
    ))
    tab_ls.append(Panel(child=p, title=case))


tabs = Tabs(tabs=tab_ls)
show(tabs)